In [1]:
# Local
import resources.image_interpolator as tools # type: ignore
import resources.dic_tools as flowTools # type: ignore
from resources.dic_Class import DicClass # type: ignore

# Math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlab

In [2]:
# Get image from file
file = 'Source/Vary Clouds (South Horizon)/#9 2024-04-08 13.05.13.png'
img1, img2 = np.hsplit(plt.imread(file), 2)

# Display Settings
np.set_printoptions(3, linewidth=200)
mlab.rc('legend', fontsize=12)
mlab.rcParams['axes.formatter.use_mathtext'] = True
plt.rc('font', family='serif', serif='cmr10', size=plt.rcParams['legend.fontsize']+4)
plt.rc('axes', unicode_minus=False)

# General
order = 2
ransac_tol = 30

In [ ]:
bigPicDic, chosen_combo, _ = flowTools.findBestGPC(img2, img1)
print(np.mean(bigPicDic.overlap_quality), chosen_combo)
bigPicDic.showOverlap()

In [ ]:
smallerDIC, chosen_combo, _ = flowTools.findBestGPC(bigPicDic.img1_overlap[300:500,500:700,:], bigPicDic.img2_overlap[300:500,500:700,:])
smallerDIC.showOverlap()
print(chosen_combo)

In [5]:
# cvFont = (cv.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv.LINE_AA) # (fontFace, fontScale, color, thickness, lineType)

# # Draw box(es) with text(s) on an OpenCV image
# def cvTextBox(img, text, loc, font=cvFont, margin=7, alpha=.4, bg=255):
#     if isinstance(text, str):
#         text, loc = [text], [loc]

#     for label, (x, y) in reversed(list(zip(text, loc))):
#         w, h = cv.getTextSize(label, *font[:2], font[3])[0] # Width, Height
#         l, t, r, b = x-margin, y+margin+3, x+w+margin, y-h-margin # Left, Top, Right, Bottom
#         sub = img[b:t,l:r]
#         img[b:t,l:r] = cv.addWeighted(sub, 1-alpha, np.ones_like(sub)*bg, alpha, 1)
#         cv.putText(img, label, (x, y), *font)

#     return img

In [ ]:
smallerDIC, chosen_combo, _ = flowTools.findBestGPC(bigPicDic.img1_overlap, bigPicDic.img2_overlap)
smallerDIC.showImages()
smallerDIC.showOverlap()

In [ ]:
smallerDIC.rollWindowAndFindBestGPC(150, 200, 200, extraWiggle=50)
flowTools.getInternalReport(smallerDIC, True)
flowTools.showInternalResults(smallerDIC)

In [ ]:
smallerDIC2 = DicClass()
smallerDIC2.defineImages(bigPicDic.img1_overlap, bigPicDic.img2_overlap)

smallerDIC2.img1_overlap = bigPicDic.img1_overlap
smallerDIC2.img2_overlap = bigPicDic.img2_overlap
smallerDIC2.windowSizeX = smallerDIC.windowSizeX
smallerDIC2.windowSizeY = smallerDIC.windowSizeY

smallerDIC2.filterMissMatched(smallerDIC, ransac_tol)
smallerDIC2.warp(order)
elsq = flowTools.error_minimization(smallerDIC2.sUV0, smallerDIC2, order, smallerDIC2.sUV0.shape)
flowTools.showWarpComparison(smallerDIC2.img1, smallerDIC2.img2, smallerDIC2.warpedImage)
flowTools.getInternalReport(smallerDIC2)
flowTools.showInternalResults(smallerDIC2)

In [ ]:
c_dis = np.array([smallerDIC2.sUV0[:,0],-smallerDIC2.sUV0[:,1]]).T
c_no_dis = np.zeros_like(c_dis)
overlapDisplacement = np.array([-bigPicDic.overlappingPoint[1],bigPicDic.overlappingPoint[0]]).reshape(2, 1)

imgs = [(tools.maxC*bigPicDic.img2/bigPicDic.img2.max()).astype(int),
        (tools.maxC*bigPicDic.img1/bigPicDic.img1.max()).astype(int)]

p01, p1, p2, img = tools.opticalFlowInterpolator(bigPicDic.img1, bigPicDic.img2, c_dis, c_no_dis, order, overlapDisplacement, imgs)
overlap = tools.refineAndConcatenateOverlaps(p01, p1, p2+overlapDisplacement, imgs)
plt.figure(figsize=(15, 15)); plt.imshow(overlap/np.max(overlap)); plt.axis('off')
plt.figure(figsize=(15, 15)); plt.imshow(img/np.max(img)); plt.axis('off')

In [ ]:
smallerDIC2.optimizeWarping(order)
print(smallerDIC2.OptimizeResult)
print(np.reshape(smallerDIC2.sUV0.ravel(), smallerDIC2.sUV0.shape))
eopt = flowTools.error_minimization(smallerDIC2.OptimizeResult.x, smallerDIC2, order, smallerDIC2.sUV0.shape)

In [ ]:
print('Least square error = %.5f' % elsq)
print('Optimization error = %.5f' % eopt)

In [ ]:
smallerDIC2.warp(optimized=True)
flowTools.showWarpComparison(smallerDIC2.img1, smallerDIC2.img2, smallerDIC2.warpedImage)

In [ ]:
C = smallerDIC2.OptimizeResult.x.reshape(smallerDIC2.sUV0.shape)
c_dis = np.array([C[:,0], -C[:,1]]).T
c_no_dis = np.zeros_like(c_dis)
if flowTools.findTriangle(c_dis.size//2) < order+1:
    c_dis = np.concatenate([c_dis, np.zeros_like(c_dis)], axis=1)
    c_no_dis = np.zeros_like(c_dis)

overlapDisplacement = np.array([-bigPicDic.overlappingPoint[1],bigPicDic.overlappingPoint[0]]).reshape(2, 1)

imgs = [(tools.maxC*bigPicDic.img2/bigPicDic.img2.max()).astype(int),
        (tools.maxC*bigPicDic.img1/bigPicDic.img1.max()).astype(int)]

p01, p1, p2, img = tools.opticalFlowInterpolator(bigPicDic.img2, bigPicDic.img1, c_dis, c_no_dis, order, overlapDisplacement, imgs)
overlap = tools.refineAndConcatenateOverlaps(p01, p1, p2+overlapDisplacement, imgs)
plt.figure(figsize=(15, 15)); plt.imshow(overlap/np.max(overlap)); plt.axis('off')
plt.figure(figsize=(15, 15)); plt.imshow(img/np.max(img)); plt.axis('off')

In [ ]:
overlap1, overlap2 = np.hsplit(overlap, 2)
evenout, chosen_combo, _ = flowTools.findBestGPC(overlap2/np.max(overlap2), overlap1/np.max(overlap1))
evenout.showOverlap()
evenout.rollWindowAndFindBestGPC(50, 75, 75, extraWiggle=50)

In [ ]:
flowTools.getInternalReport(evenout)
flowTools.showInternalResults(evenout)

In [ ]:
evenout2 = DicClass()
evenout2.defineImages(evenout.img1/np.max(evenout.img1), evenout.img2/np.max(evenout.img2))

evenout2.img1_overlap = evenout.img1_overlap
evenout2.img2_overlap = evenout.img2_overlap
evenout2.windowSizeX = evenout.windowSizeX
evenout2.windowSizeY = evenout.windowSizeY

evenout2.filterMissMatched(evenout, ransac_tol+10)
order = 3
evenout2.showQuiver()
evenout2.warp(order)
evenout2.optimizeWarping(order)
evenout2.warp(optimized=True)

flowTools.getInternalReport(evenout2)
flowTools.showWarpComparison(evenout2.img1, evenout2.img2, evenout2.warpedImage)

In [ ]:
order1 = flowTools.findTriangle(smallerDIC2.sUV0.size//2)
order2 = flowTools.findTriangle(evenout2.sUV0.size//2)

order1BiggerOrder2 = order1 > order2
sUV0Shape = smallerDIC2.sUV0.shape if order1BiggerOrder2 else evenout2.sUV0.shape

C_opt = np.zeros(sUV0Shape)
C_lsq = np.zeros(sUV0Shape)
C = np.zeros(sUV0Shape)

if smallerDIC2.OptimizeResult is not None:
    smallCs = smallerDIC2.OptimizeResult.x.reshape(smallerDIC2.sUV0.shape)
    smallCsZeros = np.zeros(sUV0Shape)
    if order1 == order2 or order1BiggerOrder2:
        smallCsZeros = smallCs
    else:
        smallCsZeros = np.array([flowTools.mapOldOrderToNewOrder(smallCs[:,0], order1, order2),
                                 flowTools.mapOldOrderToNewOrder(smallCs[:,1], order1, order2)]).T
    C_opt += smallCsZeros
elif smallerDIC2.sUV0 is not None:
    C_lsq += smallerDIC2.sUV0
if evenout2.OptimizeResult is not None:
    smallCs = evenout2.OptimizeResult.x.reshape(evenout2.sUV0.shape)
    smallCsZeros = np.zeros(sUV0Shape)
    if order1 == order2 or not order1BiggerOrder2:
        smallCsZeros = smallCs
    else:
        smallCsZeros = np.array([flowTools.mapOldOrderToNewOrder(smallCs[:,0], order2, order1),
                                 flowTools.mapOldOrderToNewOrder(smallCs[:,1], order2, order1)]).T
    C_opt += smallCsZeros
elif evenout2.sUV0 is not None:
    C_lsq += evenout2.sUV0

C += np.concatenate([C_opt[:,0,None], -C_opt[:,1,None]], axis=1)
C += np.concatenate([C_lsq[:,0,None], -C_lsq[:,1,None]], axis=1)

print('C.T =\n' + str(C.T))
c_no_dis = np.zeros_like(C)
print('overlappingPoint = ' + str(bigPicDic.overlappingPoint))
displacment = np.array([-bigPicDic.overlappingPoint[1],bigPicDic.overlappingPoint[0]]).reshape(2,1)

imgs = [(tools.maxC*bigPicDic.img2/bigPicDic.img2.max()).astype(int),
        (tools.maxC*bigPicDic.img1/bigPicDic.img1.max()).astype(int)]

p01, p1, p2, img = tools.opticalFlowInterpolator(bigPicDic.img2, bigPicDic.img1, C, c_no_dis, order, overlapDisplacement, imgs, showTranslation=True)
overlap = tools.refineAndConcatenateOverlaps(p01, p1, p2+overlapDisplacement, imgs)
plt.figure(figsize=(15, 15)); plt.imshow(overlap/np.max(overlap)); plt.axis('off')
plt.figure(figsize=(15, 15)); plt.imshow(img/np.max(img)); plt.axis('off')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8), constrained_layout=True)

A = np.mean(evenout.img2, axis=-1)
B = np.mean(evenout.img1, axis=-1)
C = np.mean(evenout2.warpedImage, axis=-1)
           
ax1.imshow(np.abs(A - B)); ax1.set_title('$|A - B|$'); ax1.set_axis_off()
ax2.imshow(np.abs(A - C)); ax2.set_title('$|A - C|$'); ax2.set_axis_off()
ax3.imshow(np.abs(np.abs(A - C) - np.abs(A - B))); ax3.set_title('$||A - B| - |A - C||$'); ax3.set_axis_off()